# Обучение модели
на основе представленных данных из архива acllmdb.zip

Процесс работы с данными состоит из этапов:
1. Импорт необходимых модулей и библиотек для предобработки;
2. Создание датафреймов;
3. Предобработка (очистка и подготовка целевой переменной);
4. Импорт библиотеки для обучения модели;
5. Обучение модели и проверка точности предсказаний;
6. Тестирование;
7. Экспорт модели.

Выбор модели классификации представлен в файле 'Отчет_Case_Lab_ML_Клинова_Мария.docx'


## Импорт необходимых модулей и библиотек для предобработки 

In [3]:
import os
import re

import pandas as pd

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [4]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\masha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\masha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\masha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\masha\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [5]:
#импорт стоп-слов английского языка
en_stops = set(stopwords.words('english')) 
print(en_stops)

{'isn', 'mightn', "don't", 'does', 'ain', 'before', 'up', "mightn't", 'your', 'this', 'are', "weren't", 'he', 'him', 'needn', "shouldn't", 'not', 'm', 'herself', 'further', 'having', 'too', 'for', 'my', 'an', 'by', 'there', 'when', 'me', 'once', 'all', 'hadn', 'will', 'it', "aren't", 'after', 'were', 'ma', 'other', "should've", 'shan', 'with', 't', 'under', 'i', 'nor', 'wouldn', 'from', 'about', 'our', 'yourselves', 'which', 'in', 'both', 'weren', 'mustn', "wasn't", "didn't", "won't", 'yours', 'ours', 'again', 'same', 'himself', 'where', 'we', 'at', 'they', 'yourself', 'on', 'only', 'don', 'any', 'd', 'theirs', 'a', 'haven', 'between', 'own', 'wasn', 'has', "mustn't", 'll', 'over', 'no', "you're", "doesn't", 'be', 'hasn', 'won', 'hers', 'o', 'against', 'aren', "she's", 'just', 'most', 'how', 's', "needn't", 'themselves', 'as', 'until', "wouldn't", "you've", 'if', 've', "shan't", 'been', 'y', 'or', "that'll", "isn't", 'couldn', 're', 'to', 'its', 'than', 'didn', 'through', 'into', "it's

## Создание датафреймов

In [7]:
cur_dir = os.getcwd()
print(os.listdir('aclImdb/train'))

['labeledBow.feat', 'neg', 'pos', 'unsup', 'unsupBow.feat', 'urls_neg.txt', 'urls_pos.txt', 'urls_unsup.txt']


In [8]:
def creating_df(type):
    """
    Создает датафрейм с колонками: 'name' (название файла), 
        'text' (текст файла) и 'sentiment' (окраска текста).
    """
    
    df = pd.DataFrame()
    def extract_rate(file_name):
        return int(file_name.split('_')[1].split('.')[0])
        
    for name in os.listdir(f'aclImdb/{type}/neg'):
        with open(f'aclImdb/{type}/neg/{name}', 'r', encoding='utf-8') as file:
            text = file.read()
            rate = extract_rate(name)
            new_row = pd.DataFrame({'name': [name], 'text': [text], 'sentiment': ['negative'], 'rate': [rate]})
            df = pd.concat([df, new_row], ignore_index=True)
    for name in os.listdir(f'aclImdb/{type}/pos'):
        with open(f'aclImdb/{type}/pos/{name}', 'r', encoding='utf-8') as file:
            text = file.read()
            rate = extract_rate(name)
            new_row = pd.DataFrame({'name': [name], 'text': [text], 'sentiment': ['positive'], 'rate': [rate]})
            df = pd.concat([df, new_row], ignore_index=True)
    df = df.sample(frac=1).reset_index(drop=True)  #перемешивает строки датафрейма
    return df

In [9]:
train = creating_df('train')

In [10]:
print(train)

              name                                               text  \
0       4847_1.txt  As usual, I am making a mad dash to see the mo...   
1      10747_4.txt  Camp Blood III is a vast improvement on Camp B...   
2       6766_3.txt  You probably heard this phrase when it come to...   
3       6565_2.txt  This movie is chilling reminder of Bollywood b...   
4       7306_3.txt  Truly one of the most dire films I've ever sat...   
...            ...                                                ...   
24995  12047_1.txt  I had the greatest enthusiasm going in to the ...   
24996   3662_9.txt  This is a really strange film--and that is NOT...   
24997  1395_10.txt  This one's a romp; many Trek fans don't rate t...   
24998   5351_7.txt  Although this film is somewhat filled with eig...   
24999   2388_3.txt  No one should ever try to adapt a Tom Robbins ...   

      sentiment  rate  
0      negative     1  
1      negative     4  
2      negative     3  
3      negative     2  
4  

## Предобработка

In [12]:
all_text = ' '.join(train['text'][:200])
unique_words = sorted(set(all_text.split()))
print(unique_words[:400])

['!"', '"', '"1"', '"24"', '"A', '"AND', '"According', '"Ah,', '"Air', '"Andrew"', '"Apart', '"Baise', '"Bevan",', '"Blair', '"Bloodrayne', '"Bloodsuckers"', '"Bottom', '"Buh-bye,', '"Bullfighter"', '"Catwoman"', '"Chad",', '"China', '"Close', '"Cradle', '"Cursed"', '"Damian\'s', '"Davitelj', '"Dead', '"Deadly', '"Deerhunter"', '"Deewar"', '"Devil', '"Didn\'t', '"Dog', '"Dragnet"', '"Dude,', '"E.T."', '"E.T.:', '"Eh?', '"Evil', '"Extension', '"Flightplan"', '"Frenchfilm"', '"From', '"Full', '"Fulltime', '"Gadsden', '"Golden', '"Golly', '"Hammer"', '"Herbie:', '"Hey', '"How', '"I', '"If', '"In', '"Invasion', '"Jenifer".', '"Kako', '"Last', '"Leatherfaces".', '"Leonard"', '"Leonora,', '"Lies"', '"Little', '"MacGyver"', '"Maratonci', '"My', '"Naked', '"National', '"New', '"No,', '"OMG', '"On', '"Osama', '"Party', '"Pilot"', '"Plan', '"Platoon".', '"Poet"', '"Projected', '"Pulp', '"Puppet', '"Putz"', '"Raiders', '"Revenge', '"Roll\'em"', '"Rondo"', '"Sabretooth"', '"Samehada', '"Saving', '

Список содержит разнообразные слова: от слов, которые могут нести определенный смылсл, до слов, которые представляют собой набор букв и цифр. Так же присуствуют знаки препинания и прочие символы, которые не несут полезной информации для анализа. Поэтому такой текст необходмо очистить для повышения точности пресказаний модели, которая будет на них обучаться.

**Очистка текстов отзывов** состоит из следующих шагов:
1. Приведение к нижнему регистру;
2. Удаление тегов <\/br>, найденных в тексте;
3. Удаление всех вспомогательных символов и пунктуации;
4. Замена всех чисел или слов, содержащих числа (например '50s'), на 'digit';
5. Удаление коротких слов из 1-2 букв;
6. Токенизация текста;
7. Лемматизация токенов;
8. Удаление стоп-слов;
9. Объединение слов обратно в строку;
10. Перемешивание строк датафрейма.

Целевая переменняая sentiment_bin необходима для сравнения методов обучения модели, выбора способа классификации и предсказания рейтинга фильма.

In [15]:
def text_cleaner(df):
    """
    Очищает текстовые данные в датафрейме
    """
    
    df['text'] = df['text'].str.lower() 
    df['text'] = df['text'].str.replace('</br>', '', regex=False)
    df['text'] = df['text'].str.replace(r'[^a-zA-Zа-я0-9\s]', '', regex=True)
    df['text'] = df['text'].str.replace(r'\b\d+\b|\b\d+\w*', 'digit', regex=True)
    df['text'] = df['text'].str.replace(r'\b\w{1,2}\b', '', regex=True)
    df['text'] = df['text'].apply(word_tokenize)
    lemmatizer = WordNetLemmatizer()
    df['text'] = df['text'].apply(lambda words: [lemmatizer.lemmatize(word) for word in words])
    df['text'] = df['text'].apply(lambda words: [word for word in words if word not in en_stops])
    df['text'] = df['text'].apply(lambda tokens: ' '.join(tokens))

    df['sentiment_bin'] = df['sentiment'].map({'positive': 1, 'negative': 0}) #создание новой целевой переменной
    
    return df

In [16]:
train = text_cleaner(train)
print(train)

              name                                               text  \
0       4847_1.txt  usual making mad dash see movie havent watched...   
1      10747_4.txt  camp blood iii vast improvement camp blood ha ...   
2       6766_3.txt  probably heard phrase come movie herbie fully ...   
3       6565_2.txt  movie chilling reminder bollywood parasite hol...   
4       7306_3.txt  truly one dire film ive ever sat ive never act...   
...            ...                                                ...   
24995  12047_1.txt  greatest enthusiasm going advance screening mo...   
24996   3662_9.txt  really strange filmand bad thing combination n...   
24997  1395_10.txt  one romp many trek fan dont rate high wellknow...   
24998   5351_7.txt  although film somewhat filled eighty cheese pl...   
24999   2388_3.txt  one ever try adapt tom robbins book screen mov...   

      sentiment  rate  sentiment_bin  
0      negative     1              0  
1      negative     4              0  
2     

Заново просмотрим набор уникальных слов из первых 200 строк датафрейма. Теперь данные очищены, это повысит точность модели.

In [18]:
all_text = ' '.join(train['text'][:200])
unique_words = sorted(set(all_text.split()))
print(unique_words[:400])

['abandoned', 'abc', 'abducted', 'ability', 'abilitytalent', 'able', 'aboard', 'abomination', 'abort', 'aboutbr', 'aboveground', 'abovementioned', 'abovethelaw', 'abraham', 'absolute', 'absolutely', 'absurd', 'absurdity', 'abusing', 'abusive', 'academy', 'accelerating', 'accent', 'acceptable', 'acceptance', 'accepted', 'access', 'accessible', 'accident', 'accidental', 'accidentally', 'acclaim', 'acclaimed', 'accompanied', 'accomplishes', 'accomplishment', 'according', 'account', 'accurately', 'accused', 'ace', 'acharya', 'achieve', 'achievement', 'achieving', 'achingly', 'acknowledged', 'across', 'act', 'acted', 'acting', 'actingbr', 'action', 'actionsbr', 'activity', 'actor', 'actoractress', 'actorwriterdirectors', 'actress', 'actual', 'actually', 'ad', 'adam', 'adapt', 'adaptation', 'adaption', 'add', 'added', 'addictbr', 'adding', 'additional', 'additionally', 'address', 'adequate', 'adjusted', 'administer', 'administration', 'administrative', 'administrator', 'admirably', 'admire',

In [19]:
train.to_csv('cleaned_train_data.csv') #сохранение очищенных данных в .csv файл 

Аналогичные действия с данными для тестирования:

In [21]:
test = creating_df('test')
test = text_cleaner(test)

In [22]:
print(test)

              name                                               text  \
0       4989_2.txt  let start saying never wanted see movie first ...   
1        678_1.txt  wa wondering possessed organizer victoria film...   
2      11258_8.txt  wasnt planning watching wasted saw mtv preview...   
3       405_10.txt  note writing review see listing indeed series ...   
4       2074_1.txt  like guy said sux count word said entire movie...   
...            ...                                                ...   
24995   1272_8.txt  great british director christopher nolan momen...   
24996   5165_2.txt  wonderful image good intention come yet anothe...   
24997   5810_1.txt  hard know exactly say ever bland dull little f...   
24998   2733_3.txt  terry cunningham directs scifi network origina...   
24999  6724_10.txt  saw korean version daisy first came across sim...   

      sentiment  rate  sentiment_bin  
0      negative     2              0  
1      negative     1              0  
2     

In [23]:
test.to_csv('cleaned_test_data.csv') #сохранение очищенных данных в .csv файл 

## Импорт библиотек для обучения модели

In [25]:
import sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [26]:
import numpy as np

In [27]:
print(sklearn.__version__)

1.5.1


## Обучение моделей и проверка точности предсказаний

Для предсказания рейтинга фильма рассмотрим **два способа**:
1. Мультиклассовая классификация с целью предсказания рейтинга фильма, целевые значения - train['rate'];

In [30]:
#пайплайн последовательно векторизует числа, после этого применяет алгорим классификации
pipe_text_multiclass_classfication = Pipeline([
    ('vectorizer', TfidfVectorizer(ngram_range=(1, 3), max_features=20000)),
    ('pca', PCA(n_components=1000)),
    ('classifier', LogisticRegression(max_iter=1000, random_state = 42))
])

#разделение данных
features_train, features_test, target_train, target_test = train['text'], test['text'], train['rate'], test['rate']

pipe_text_multiclass_classfication.fit(features_train, target_train)
prediction = pipe_text_multiclass_classfication.predict(features_test)

#проверка точности
accuracy_multiclass = accuracy_score(target_test, prediction)

In [31]:
print(accuracy_multiclass)

0.42272


2. Бинарная классификация отзывов на положительные и негативные, рейтинг фильма определяется как вероятность принадлежности к классу "positive", с округление в большую сторону, целевые значения - train['sentiment_bin'].

In [33]:
#пайплайн последовательно векторизует числа, после этого применяет алгорим классификации
pipe_text_binary_classification = Pipeline([
    ('vectorizer', TfidfVectorizer(ngram_range=(1, 3), max_features=10000)), 
    ('classifier', LogisticRegression(max_iter=5000))
])

#разделение данных
features_train, features_test, target_train, target_test = train['text'], test['text'], train['sentiment_bin'], test['sentiment_bin']

pipe_text_binary_classification.fit(features_train, target_train)
prediction = pipe_text_binary_classification.predict(features_test)

#проверка точности
accuracy_binary = accuracy_score(target_test, prediction)

In [34]:
print(accuracy_binary)

0.88472


## Тестирование

На основе датафрейма test, предскажем рейтинг фильма мультиклассовой моделью, окраску текста бинарной моделью и рейтинг на осонове вероятности.

In [37]:
test['predicted_rate_multiclass'] = pipe_text_multiclass_classfication.predict(test['text'])
test['predicted_semtiment_binary'] = pipe_text_binary_classification.predict(test['text'])
test['probability_rate'] = np.ceil(pipe_text_binary_classification.predict_proba(test['text'])[:, 1] * 10)

In [38]:
print(test.head(10))

          name                                               text sentiment  \
0   4989_2.txt  let start saying never wanted see movie first ...  negative   
1    678_1.txt  wa wondering possessed organizer victoria film...  negative   
2  11258_8.txt  wasnt planning watching wasted saw mtv preview...  positive   
3   405_10.txt  note writing review see listing indeed series ...  positive   
4   2074_1.txt  like guy said sux count word said entire movie...  negative   
5  4404_10.txt  thank goodness coen brother success ha brought...  positive   
6      5_7.txt  went see movie always liked kevin costner felt...  positive   
7   4995_1.txt  cant give le star tried moment sure halfway st...  negative   
8  5789_10.txt  always knew day wa coming knew much oil ground...  positive   
9   9971_9.txt  saw toronto international film festival beauti...  positive   

   rate  sentiment_bin  predicted_rate_multiclass  predicted_semtiment_binary  \
0     2              0                          1

In [39]:
print('Процент соответствия реального рейтинга с предсказанным мультиклассовой моделью:', 
      (test['rate'] == test['predicted_rate_multiclass']).mean() * 100)
print('Процент соответствия реального рейтинга вероятности принадлежности классу "positive"', 
      (test['rate'] == test['probability_rate']).mean() * 100)

Процент соответствия реального рейтинга с предсказанным мультиклассовой моделью: 42.272
Процент соответствия реального рейтинга вероятности принадлежности классу "positive" 28.92


In [40]:
print('Процент соответствия реальной окраски текста на соснове предсказаний мультиклассовой модели:', 
      (test['sentiment_bin'] == np.where(test['predicted_rate_multiclass'] > 5, 1, 0)).mean() * 100)
print('Процент соответствия реальной окраски текста и пресказанной бинарной моделью:', 
      (test['sentiment_bin'] == test['predicted_semtiment_binary']).mean() * 100)

Процент соответствия реальной окраски текста на соснове предсказаний мультиклассовой модели: 87.20400000000001
Процент соответствия реальной окраски текста и пресказанной бинарной моделью: 88.472


Маленькая точность мультклассовой модели обусловлена многоклассовостью (классы от 1 до 10) и субъективностью оценок. По отзыву точно определить рейтинг фильма невозможно, так, например, негативный отзыв может иметь оценку 2, а модель может предсказать любое достаточно близкое значение (1 или 3).

## Экспорт модели

Для создания сервиса импортируем модель мультиклассовой классификации, т.к. она имеет наиболее точные предсказания.

In [44]:
import joblib

In [45]:
joblib.dump(pipe_text_multiclass_classfication, 'text_classification_model.pkl')

['text_classification_model.pkl']